# Usage of `src.analysis.weather_interpolator` module.

This notebook outlies the basic usage of the `src.analysis.weather_interpolator` module. Used to interpolate weather conditions at any time, lon, lat, and elevation.
 
**Requirements**
 - A csv with weather station data for the whole country for a given time interval
 
**Helpful Links**


## Basic Setup

In [42]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
#
import sqlite3

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

%load_ext autoreload
%autoreload 2

root_path = os.path.normpath(root_path) # Path from this notebook to the root directory
config_path_from_root = os.path.normpath('config/config_tutorial.yml') # Path from root to the desired config file
config_path = os.path.join(root_path, config_path_from_root) # Defining path from this notebook to config file

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

# Creates an instance of the weather interpolator
interpolator = weather_interpolator.WeatherInterpolator(config, logger=logger)

# Defining location of data
flights_database = '../data/flight/KDEN_KSEA_2023-01-01_2023-01-31.sqlite'
weather_database = '../data/weather/1673827200_1685923200.sqlite'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Interpolating weather at specific time, lat, lon, and elevation

By Default, the interpolator will look for data in the weather's out-dir as specified in the config file, unless it's given one directly.

The interpolator will also calibrate the weather data, unless specified that it has already beed calibrated as an argument

Weather model calibration happens inside the estimation function.

Target needs to be a dictionary with the necessary parameters, in this case we'll use the avegate values form the weather stations info

## Estimating scalar at given target location and time

In [24]:
flights_connection = sqlite3.connect(flights_database)
weather_connection = sqlite3.connect(weather_database)

flight_id = pd.read_sql_query("SELECT flight_id FROM flights ORDER BY RANDOM() LIMIT 1;", flights_connection).values[0, 0]

mean_time, max_time, min_time = pd.read_sql_query(f"""
                                SELECT AVG(time) as avg_time, MAX(time) as max_time, MIN(time) as min_time
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]

mean_latitude, max_latitude, min_latitude = pd.read_sql_query(f"""
                                SELECT AVG(lat) as avg_lat, MAX(lat) as max_lat, MIN(lat) as min_lat
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]

mean_longitude, max_longitude, min_longitude = pd.read_sql_query(f"""
                                SELECT AVG(lon) as avg_lon, MAX(lon) as max_lon, MIN(lon) as min_lon
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]

mean_geoaltitude = pd.read_sql_query(f"""
                                SELECT AVG(geoaltitude) as avg_geoaltitude
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0, 0]

target = {'lon': mean_longitude,
         'lat': mean_latitude,
         'time': mean_time,
         'elevation': mean_geoaltitude,
         }

time_thresh = config['statistics']['interpolation']['weather']['time-thresh']
min_time -= time_thresh
max_time += time_thresh

latitude_range = max_latitude - min_latitude
longitude_range = max_longitude - min_longitude

max_latitude += latitude_range*0.1
min_latitude -= latitude_range*0.1
max_longitude += longitude_range*0.1
min_longitude -= longitude_range*0.1

flight_weather_data = pd.read_sql_query(f"""
                                SELECT ws.lat, ws.lon, ws.elevation, ws.sigma, wd.*
                                FROM weather_data as wd
                                JOIN weather_stations as ws ON ws.station_id = wd.station_id
                                WHERE wd.time BETWEEN {min_time} AND {max_time}
                                AND ws.lat BETWEEN {min_latitude} AND {max_latitude}
                                AND ws.lon BETWEEN {min_longitude} AND {max_longitude};
                               """,
                               weather_connection
                                )

state_vectors = pd.read_sql_query(f"""
                                SELECT DISTINCT state_vectors.*
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                               flights_connection)
# query

flights_connection.close()
weather_connection.close()

In [25]:
len(flight_weather_data)

13976

In [28]:
%%time
interpolator.estimate_scalars(target, ['tmpf'], stations_data=flight_weather_data)


CPU times: user 257 ms, sys: 6.72 ms, total: 264 ms
Wall time: 279 ms


array([-102.05703819])

In [29]:
%%time
interpolator.estimate_scalars(target, ['air_pressure'], stations_data=flight_weather_data)


CPU times: user 242 ms, sys: 4.91 ms, total: 247 ms
Wall time: 251 ms


array([192.43300048])

In [30]:
%%time
interpolator.estimate_scalars(target, ['air_density'], stations_data=flight_weather_data)


CPU times: user 263 ms, sys: 5.77 ms, total: 269 ms
Wall time: 303 ms


array([0.33742873])

In [31]:
%%time
interpolator.estimate_scalars(target, ['clouds'], stations_data=flight_weather_data)


CPU times: user 287 ms, sys: 7.24 ms, total: 294 ms
Wall time: 305 ms


array([7.13819534e-06])

In [32]:
%%time
interpolator.estimate_scalars(target, ['severity'], stations_data=flight_weather_data)


CPU times: user 257 ms, sys: 6.56 ms, total: 264 ms
Wall time: 269 ms


array([0.])

In [92]:
%%time
target = {
    'lon': state_vectors.iloc[0]['lon'],
    'lat': state_vectors.iloc[0]['lat'],
    'time': state_vectors.iloc[0]['time'],
    'elevation': state_vectors.iloc[0]['geoaltitude']
}
interpolator.estimate_scalars(target, ['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity'], stations_data=flight_weather_data)


CPU times: user 607 ms, sys: 4.8 ms, total: 612 ms
Wall time: 616 ms


array([1.89259415e+01, 8.18128781e+02, 1.07193333e+00, 2.00000000e-01,
       8.32240000e+00, 0.00000000e+00])

In [96]:
%%time
state_vectors = interpolator.compute_flight_weather_quantities(['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity'], state_vectors, stations_data=flight_weather_data, debug=False)
state_vectors


CPU times: user 1min 24s, sys: 618 ms, total: 1min 25s
Wall time: 1min 26s


,vector_id,flight_id,time,time_normalized,lat,lon,geoaltitude,baroaltitude,heading,velocity,tmpf,air_pressure,air_density,clouds,sknt,severity
0,140988,a44cfd_1674567430_1674576256_KDEN_KSEA,1674567432,0,39.883026,-104.696005,1722.120000,1737.360000,1.930587,91.623040,18.925941,818.128781,1.071933,0.200000,8.322400,0.0
1,140989,a44cfd_1674567430_1674576256_KDEN_KSEA,1674567433,1,39.883791,-104.695963,1743.973096,1758.350406,2.071287,90.719268,18.767807,816.830596,1.070559,0.200241,8.564206,0.0
2,140990,a44cfd_1674567430_1674576256_KDEN_KSEA,1674567434,2,39.884556,-104.695921,1765.826192,1779.340811,2.211986,89.815496,18.609672,815.532412,1.069185,0.200482,8.806011,0.0
3,140991,a44cfd_1674567430_1674576256_KDEN_KSEA,1674567435,3,39.885320,-104.695879,1787.679288,1800.331217,2.352685,88.911724,18.451538,814.234228,1.067811,0.200722,9.047817,0.0
4,140992,a44cfd_1674567430_1674576256_KDEN_KSEA,1674567436,4,39.886085,-104.695836,1809.532384,1821.321623,2.493385,88.007953,18.293403,812.936044,1.066437,0.200963,9.289622,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,149808,a44cfd_1674567430_1674576256_KDEN_KSEA,1674576252,8820,47.455540,-122.317989,102.510762,-69.298477,180.504458,59.202269,39.530467,1000.550482,1.256837,0.720000,7.810215,0.0
8821,149809,a44cfd_1674567430_1674576256_KDEN_KSEA,1674576253,8821,47.455001,-122.317992,101.648071,-71.023858,180.517406,57.649259,39.530467,1000.550482,1.256837,0.720000,7.810215,0.0
8822,149810,a44cfd_1674567430_1674576256_KDEN_KSEA,1674576254,8822,47.454462,-122.317995,100.785381,-72.749238,180.530355,56.096249,39.530467,1000.550482,1.256837,0.720000,7.810215,0.0
8823,149811,a44cfd_1674567430_1674576256_KDEN_KSEA,1674576255,8823,47.453923,-122.317998,99.922690,-74.474619,180.543304,54.543239,39.530467,1000.550482,1.256837,0.720000,7.810215,0.0


In [98]:
len(flight_ids)*(1*60 + 24)/60/60

15.143333333333334

## Looping thorugh flight_ids and computing weather

This code also creates a new state_vector_weather table to save the values

In [10]:
flights_connection = sqlite3.connect(flights_database)
weather_connection = sqlite3.connect(weather_database)

flight_ids = pd.read_sql_query("SELECT flight_id FROM flights;", flights_connection).values[:,0]
time_thresh = config['statistics']['interpolation']['weather']['time-thresh']

new_columns = ['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity']

cursor = flights_connection.cursor()

# Create the new table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS state_vector_weather (
        vector_id INTEGER PRIMARY KEY,
        tmpf REAL, 
        air_pressure REAL, 
        air_density REAL, 
        clouds REAL, 
        sknt REAL,
        severity REAL,
    );
''')
flights_connection.commit()

for i, flight_id in enumerate(flight_ids):
    clear_output(wait=True)
    print(f'{flight_id} | {i}/{len(flight_ids)}')

    print('Loading time limits')
    max_time, min_time = pd.read_sql_query(f"""
                                SELECT MAX(time) as max_time, MIN(time) as min_time
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]
    
    min_time -= time_thresh
    max_time += time_thresh

    print('Loading relevant weather data')
    flight_weather_data = pd.read_sql_query(f"""
                                    SELECT ws.lat, ws.lon, ws.elevation, ws.sigma, wd.*
                                    FROM weather_data as wd
                                    JOIN weather_stations as ws ON ws.station_id = wd.station_id
                                    WHERE wd.time BETWEEN {min_time} AND {max_time};
                                   """,
                                   weather_connection
                                    )

    print('Loading state vectors')
    state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """,
                                   flights_connection)
    
    print('Computing weather values')
    state_vectors = interpolator.compute_flight_weather_quantities(['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity'], state_vectors, stations_data=flight_weather_data)
    
    print('Adding new columns')
    for col in new_columns:
        try:
            print(f"Attempting to add column '{col}' to 'state_vectors'.")
            cursor.execute(f"ALTER TABLE state_vectors ADD COLUMN {col} REAL;")
            flights_connection.commit()
            print(f"Column '{col}' added successfully.")
        except sqlite3.OperationalError as e:
            print(f"Error adding column '{col}': {e}")
            # If the error message is not about the column existing, re-raise the exception
            if not 'duplicate column name' in str(e).lower():
                raise
            else:
                print(f"Column '{col}' already exists.")
    
    print("Adding newly calculated values")
    for index, row in state_vectors.iterrows():
        insert_query = """
            INSERT INTO state_vector_weather (vector_id, tmpf, air_pressure, air_density, clouds, sknt, severity) 
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ON CONFLICT(vector_id) DO UPDATE SET
            tmpf = excluded.tmpf, 
            air_pressure = excluded.air_pressure,
            air_density = excluded.air_density, 
            clouds = excluded.clouds, 
            sknt = excluded.sknt,
            severity = excluded.severity;
        """
        cursor.execute(insert_query, (row['vector_id'], row['tmpf'], row['air_pressure'], row['air_density'], row['clouds'], row['sknt'], row['severity']))
    flights_connection.commit()
# Commit the transaction after all updates

flights_connection.close()
weather_connection.close()
# Save database that now has the new columns somehow


a5dee3_1675198135_1675207118_KDEN_KSEA | 648/649
Loading time limits
Loading relevant weather data
Loading state vectors
Computing weather values
Adding new columns
Attempting to add column 'tmpf' to 'state_vectors'.
Error adding column 'tmpf': duplicate column name: tmpf
Column 'tmpf' already exists.
Attempting to add column 'air_pressure' to 'state_vectors'.
Error adding column 'air_pressure': duplicate column name: air_pressure
Column 'air_pressure' already exists.
Attempting to add column 'air_density' to 'state_vectors'.
Error adding column 'air_density': duplicate column name: air_density
Column 'air_density' already exists.
Attempting to add column 'clouds' to 'state_vectors'.
Error adding column 'clouds': duplicate column name: clouds
Column 'clouds' already exists.
Attempting to add column 'sknt' to 'state_vectors'.
Error adding column 'sknt': duplicate column name: sknt
Column 'sknt' already exists.
Adding newly calculated values


In [35]:
def Integrate_Clouds(state_vectors):
    return np.sum(state_vectors['clouds'].values)/(state_vectors['time'].values[-1] - state_vectors['time'].values[0])

def Integrate_Wind_Speed(state_vectors):
    return np.sum(state_vectors['sknt'].values)/(state_vectors['time'].values[-1] - state_vectors['time'].values[0])

def Integrate_Path_Lenght(state_vectors):
    # Computes xy displacements
    dxy = np.array([utils.haversine_distance(row_a.lat, row_a.lon, row_b.lat, row_b.lon) for row_a, row_b in zip(state_vectors[:-1].itertuples(), state_vectors[1:].itertuples())])
    # Computes z displacement
    dz = np.array([np.abs(row_a.geoaltitude - row_b.geoaltitude) for row_a, row_b in zip(state_vectors[:-1].itertuples(), state_vectors[1:].itertuples())])
    # Compute displacements
    dr = np.sqrt(dxy**2 + dz**2)
    
    # Integrates
    return np.sum(dr)

def Integrate_Time(state_vectors):
    return (state_vectors['time'].values[-1] - state_vectors['time'].values[0])


for i, flight_id in enumerate(flight_ids):
    flights_connection = sqlite3.connect(flights_database)

    state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT sv.*, svw.tmpf, svw.air_pressure, svw.air_density, svw.clouds, svw.sknt
                                    FROM state_vectors AS sv
                                    JOIN flights ON flights.flight_id = sv.flight_id
                                    LEFT JOIN state_vector_weather AS svw ON sv.vector_id = svw.vector_id
                                    WHERE sv.flight_id = "{flight_id}";
                                   """,
                                   flights_connection).dropna(axis = 'columns')

    flights_connection.close()

    integral = Integrate_Path_Lenght(state_vectors)

    print(f'{flight_id} | {i}/{len(flight_ids)} | Path integral: {integral}')

ada167_1672524657_1672534041_KDEN_KSEA | 0/649 | Path integral: 1739167.9731069868
a923a8_1672527139_1672536531_KDEN_KSEA | 1/649 | Path integral: 1725696.8988892848
aacd50_1672541456_1672550735_KDEN_KSEA | 2/649 | Path integral: 1737301.2050031873
abb19c_1672549350_1672558473_KDEN_KSEA | 3/649 | Path integral: 1751629.8954189336
a3e55a_1672598684_1672607288_KDEN_KSEA | 4/649 | Path integral: 1751916.8578521956
a326d1_1672601433_1672609895_KDEN_KSEA | 5/649 | Path integral: 1731400.854510125
a01037_1672602317_1672611355_KDEN_KSEA | 6/649 | Path integral: 1780267.1108377439
ab1644_1672603218_1672611926_KDEN_KSEA | 7/649 | Path integral: 1739448.2215336878
a2b8c5_1672606449_1672615130_KDEN_KSEA | 8/649 | Path integral: 1735708.6592100689
ad01ab_1672753383_1672761742_KDEN_KSEA | 9/649 | Path integral: 1762622.728310032
a34e9d_1672755010_1672763243_KDEN_KSEA | 10/649 | Path integral: 1737486.898148146
a6ca7f_1672756152_1672764404_KDEN_KSEA | 11/649 | Path integral: 1735036.1718804687
a4232

KeyboardInterrupt: 

,vector_id,flight_id,time,time_normalized,lat,lon,geoaltitude,baroaltitude,heading,velocity,tmpf,air_pressure,air_density,clouds,sknt
0,438157,ab1644_1672603218_1672611926_KDEN_KSEA,1672603219,0,39.841003,-104.730875,1783.080000,1851.660000,270.934088,94.670277,32.428939,816.808499,1.040852,0.0,9.699912
1,438158,ab1644_1672603218_1672611926_KDEN_KSEA,1672603220,1,39.841055,-104.731961,1796.571434,1864.277152,271.644405,94.753353,32.330537,815.976322,1.039990,0.0,9.868075
2,438159,ab1644_1672603218_1672611926_KDEN_KSEA,1672603221,2,39.841106,-104.733047,1810.062869,1876.894303,272.354723,94.836429,32.232134,815.144145,1.039127,0.0,10.036238
3,438160,ab1644_1672603218_1672611926_KDEN_KSEA,1672603222,3,39.841157,-104.734134,1823.554303,1889.511455,273.065041,94.919504,32.133732,814.311967,1.038264,0.0,10.204401
4,438161,ab1644_1672603218_1672611926_KDEN_KSEA,1672603223,4,39.841208,-104.735220,1837.045738,1902.128607,273.775359,95.002580,32.035329,813.479790,1.037402,0.0,10.372564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8703,446860,ab1644_1672603218_1672611926_KDEN_KSEA,1672611922,8703,47.460534,-122.317904,107.048442,66.334345,180.428573,68.810685,44.918627,999.385073,1.241971,0.0,4.211754
8704,446861,ab1644_1672603218_1672611926_KDEN_KSEA,1672611923,8704,47.459925,-122.317904,105.051331,63.085758,180.430771,68.456546,44.918627,999.385073,1.241971,0.0,4.211754
8705,446862,ab1644_1672603218_1672611926_KDEN_KSEA,1672611924,8705,47.459316,-122.317904,103.054221,59.837172,180.432969,68.102406,44.918627,999.385073,1.241971,0.0,4.211754
8706,446863,ab1644_1672603218_1672611926_KDEN_KSEA,1672611925,8706,47.458707,-122.317904,101.057110,56.588586,180.435166,67.748267,44.918627,999.385073,1.241971,0.0,4.211754


In [ ]:
# %%time
# scalars = ['tmpf', 'air_pressure', 'air_density', 'clouds']
# scalar_values = {scalar: np.repeat(np.nan, len(state_vectors)) for scalar in scalars}
# step = config['statistics']['interpolation']['flights']['step']
# for i, row in state_vectors.iloc[::step].iterrows():
#     clear()
#     print(f'{i}/{len(state_vectors)}')
#     target = {
#         'lon': row['lon'],
#         'lat': row['lat'],
#         'timestamp': row['time'],
#         'elevation': row['geoaltitude'],
#          }
#     values = interpolator.estimate_scalars(target, scalars, stations_data=all_stations_data)
#     for j, scalar in enumerate(scalars):
#         scalar_values[scalar][i] = values[j]
# for scalar in scalars:
#     state_vectors[scalar] = scalar_values[scalar]
#     state_vectors[scalar] = state_vectors[scalar].interpolate(method='linear')

# state_vectors

In [ ]:
# compressed = pd.read_csv('tutorial_data/state_vectors_compressed.csv', index_col = 0)
# compressed['geolatitude'].plot()

In [ ]:
# import os
# large_df = None
# path = '../data/flight/KDEN_KSEA/state_vectors/'
# files = [path + f for f in os.listdir(path) if f.endswith('.csv')]
# for file in files:
#     temp_df = pd.read_csv(file, index_col = 0)
#     temp_df['icao24'] = [file.split('/')[-1].split('_')[0]]*len(temp_df)
#     if large_df is None:
#         large_df = temp_df.copy()
#     else:
#         large_df = pd.concat([large_df, temp_df.copy()])


In [ ]:
# large_df.to_csv('../data/flight/KDEN_KSEA/all_flights.csv')

In [ ]:
# state_vectors

In [ ]:
# flights_connection = sqlite3.connect(flights_database)
# cursor = flights_connection.cursor()

# # Create a new table with unique entries
# cursor.execute('''
# CREATE TABLE unique_state_vectors AS 
# SELECT * FROM state_vectors 
# GROUP BY vector_id;
# ''')

# flights_connection.commit()

# # Drop the old table
# cursor.execute('DROP TABLE state_vectors;')

# # Rename the new table to the original name
# cursor.execute('ALTER TABLE unique_state_vectors RENAME TO state_vectors;')

# flights_connection.commit()

# flights_connection.close()

In [ ]:
# flights_connection.commit()

In [ ]:
# flights_connection = sqlite3.connect(flights_database)
# cursor = flights_connection.cursor()

# # SQL to create a new table with unique entries, keeping the most recent row for each vector_id
# cursor.execute('''
# CREATE TABLE unique_state_vectors AS 
# SELECT * 
# FROM state_vectors 
# WHERE rowid IN (
#     SELECT MAX(rowid) 
#     FROM state_vectors 
#     GROUP BY vector_id
# );
# ''')

# flights_connection.commit()

# # Drop the old table
# cursor.execute('DROP TABLE state_vectors;')

# # Rename the new table to the original name
# cursor.execute('ALTER TABLE unique_state_vectors RENAME TO state_vectors;')

# flights_connection.commit()
# flights_connection.close()
